In [6]:
import pandas as pd 
import numpy as np
data_OR=pd.read_excel('Score_OR_valid.xlsx')
data_OR.head()

,Chinese,English,No.1,No.2,No.3,Num_GWAS,Num_Match_SNP,SNP_info
0,黑色素瘤,Melanoma,0.000000,0.275050,0.000000,73,4,"{'rs401681': [['A', 0.1823215567939546]], 'rs4..."
1,慢性淋巴细胞白血病,Chronic lymphocytic leukemia,0.414431,0.501671,0.519456,121,16,"{'rs674313': [['T', 0.5247285289349821]], 'rs3..."
2,成神经细胞瘤,Neuroblastoma,0.500000,0.500000,1.000000,10,1,"{'rs9295536': [['A', 0.30527638085273207]]}"
3,尤文肉瘤,Ewing sarcoma,0.500000,1.000000,1.000000,7,1,"{'rs9430161': [['G', 0.7884573603642703]]}"
4,基底细胞癌,Basal cell carcinoma,0.211996,0.250663,0.303299,140,11,"{'rs401681': [['C', 0.20701416938432612]], 'rs..."


# 风险等级函数


In [7]:
#风险对应表--1（文字版）
def Risk_Level_1(RR):
    if RR<0.5:
        return '显著低于一般人群'
    elif RR<0.9:
        return '低于一般人群'
    elif RR<1.1:
        return '相当于一般人群'
    elif RR<2.0:
        return '高于一般人群'
    else:
        return '显著高于一般人群'

In [8]:
#风险对应表--2（等级版）
def Risk_Level_2(RR):
    if RR<0.5:
        return '1'
    elif RR<0.9:
        return '2'
    elif RR<1.1:
        return '3'
    elif RR<2.0:
        return '4'
    else:
        return '5'

# 基础计算

In [14]:
from math import nan
#计算群体风险均值
RR_avg=[]
for index, row in data_OR.iterrows():
    RR_avg.append(np.mean([row['No.1'], row['No.2'], row['No.3']]))

#准备数据，计算RR
output_data=pd.DataFrame(data_OR,columns=['Chinese', 'English', 'No.1', 'SNP_info'])
output_data=output_data.rename(columns={'No.1':'risk score'})
RR_list=[]
for index, row in output_data.iterrows():
    if RR_avg[index]==0:
        RR_list.append(nan)
    else:
        RR_list.append(row['risk score']/RR_avg[index])
output_data['risk_ratio']=RR_list
#删去RR==0的值，删去风险分数列
output_data=output_data.drop(output_data[output_data['risk_ratio'].isna()].index)
output_data=pd.DataFrame(output_data,columns=['Chinese', 'English', 'risk_ratio', 'SNP_info'])
#按照RR排序
output_data=output_data.sort_values(by=['risk_ratio'], ascending=False)
output_data.reset_index(drop=True,inplace=True)

In [16]:
output_data.head(5)

,Chinese,English,risk_ratio,SNP_info
0,进行性核上性麻痹,Progressive supranuclear palsy,3.000000,"{'rs242557': [['G', 0.6450592697528901]]}"
1,赌博倾向,Pathological gambling,3.000000,"{'rs7591351': [['T', 0.5128236264286637]]}"
2,自闭症,Autism,2.860524,"{'rs10038113': [['T', 0.28517894223366247]], '..."
3,胰腺癌,Pancreatic cancer,1.671239,"{'rs2039553': [['A', 0.5481214085096876]], 'rs..."
4,强直性脊柱炎,Ankylosing spondylitis,1.625531,"{'rs10865331': [['A', 0.23901690047049992]], '..."


In [17]:
output_data.shape

(93, 4)

# 输出三个版本的数据    
1. 风险比率（output_data）
2. 文字版描述（output_data_2）
3. 1-5个等级（output_data_3）

In [18]:
#文字版描述（output_data_2）
output_data_2=output_data.copy()
for index in range(output_data_2.shape[0]):
    try:
        output_data_2['risk_ratio'][index]=Risk_Level_1(output_data_2['risk_ratio'][index])
    except:
        print(index)
output_data_2.head()

D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
D:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,Chinese,English,risk_ratio,SNP_info
0,进行性核上性麻痹,Progressive supranuclear palsy,显著高于一般人群,"{'rs242557': [['G', 0.6450592697528901]]}"
1,赌博倾向,Pathological gambling,显著高于一般人群,"{'rs7591351': [['T', 0.5128236264286637]]}"
2,自闭症,Autism,显著高于一般人群,"{'rs10038113': [['T', 0.28517894223366247]], '..."
3,胰腺癌,Pancreatic cancer,高于一般人群,"{'rs2039553': [['A', 0.5481214085096876]], 'rs..."
4,强直性脊柱炎,Ankylosing spondylitis,高于一般人群,"{'rs10865331': [['A', 0.23901690047049992]], '..."


In [19]:
#1-5个等级（output_data_3）
output_data_3=output_data.copy()
for index in range(output_data_3.shape[0]):
    try:
        output_data_3['risk_ratio'][index]=Risk_Level_2(output_data_3['risk_ratio'][index])
    except:
        print(index)
output_data_3.head()

D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


,Chinese,English,risk_ratio,SNP_info
0,进行性核上性麻痹,Progressive supranuclear palsy,5.0,"{'rs242557': [['G', 0.6450592697528901]]}"
1,赌博倾向,Pathological gambling,5.0,"{'rs7591351': [['T', 0.5128236264286637]]}"
2,自闭症,Autism,5.0,"{'rs10038113': [['T', 0.28517894223366247]], '..."
3,胰腺癌,Pancreatic cancer,4.0,"{'rs2039553': [['A', 0.5481214085096876]], 'rs..."
4,强直性脊柱炎,Ankylosing spondylitis,4.0,"{'rs10865331': [['A', 0.23901690047049992]], '..."


# 输出到excel

In [20]:
output_data.to_excel('D:/2_Professional Courses/132_基因项目/新模型/旧数据最终的输出结果/output1.xls',index=False)
output_data_2.to_excel('D:/2_Professional Courses/132_基因项目/新模型/旧数据最终的输出结果/output2.xls',index=False)
output_data_3.to_excel('D:/2_Professional Courses/132_基因项目/新模型/旧数据最终的输出结果/output3.xls',index=False)

In [24]:
# 输出群体的RR平均值
Group_RR=pd.DataFrame(data_OR, columns=['Chinese', 'English'])
Group_RR['RR_avg']=RR_avg
Group_RR.to_excel('RR_avg.xls')

In [22]:
Group_RR.head()

,Chinese,English,RR_avg
0,黑色素瘤,Melanoma,0.091683
1,慢性淋巴细胞白血病,Chronic lymphocytic leukemia,0.478519
2,成神经细胞瘤,Neuroblastoma,0.666667
3,尤文肉瘤,Ewing sarcoma,0.833333
4,基底细胞癌,Basal cell carcinoma,0.255319


In [2]:
a=[1,2,3,4]
type(str(len(a)))

str